<a href="https://colab.research.google.com/github/mrzaid/ml-bootcamp/blob/main/pysparkMLLIB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# !pip install pyspark
# !pip install findspark 
# 1. nstall all the dependencies in Colab environment i.e. Apache Spark 2.4.4 with hadoop 2.7, Java 8 and Findspark to locate the spark in the system
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

tar: spark-2.4.4-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [10]:

# 2. Setup Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [11]:
# 3. Start Spark Session
import findspark #spark ko mangwa rahay hou
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Exception: ignored

In [ ]:
df=spark.createDataFrame([{"spark":"jupyter"} for x in range(1000) ])
df.show(10)

In [ ]:
from google.colab import files
files.upload()

In [ ]:
data=spark.read.csv("Student_Grades_Data.csv",header=True,inferSchema=True)

In [ ]:
data.printSchema()

In [ ]:
data.show()

In [ ]:
# create feat array omit last column
feature_cols=data.columns[:-1]
from pyspark.ml.feature import VectorAssembler
vect_assembler=VectorAssembler(inputCols=feature_cols,outputCol='features')
# utilize assembler created above to add feat column
data_w_features=vect_assembler.transform(data)
data_w_features.show(4)

In [ ]:
from pyspark.ml.feature import StringIndexer,OneHotEncoderEstimator,VectorAssembler
stages=[]
for categoricalCol in categorical_cols:
  stringIndexer=StringIndexer(inputCol=categoricalCol,outputCol=categoricalCol+'Index')
  Ohencoder=OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()],outputCols=[categoricalCol+'_catVec'])
stages +=[stringIndexer,Ohencoder]
assemblerInputs=[c+'_catVec' for c in categorical_cols] + numerical_cols
Vectassembler=VectorAssembler(inputCols=assemblerInputs,outputCol='features')
stages += [Vectassembler]

In [ ]:
from pyspark.ml import Pipeline
cols=data.columns
pipeline=Pipeline(stages=stages)
pipelineModel=pipeline.fit(data)
data=pipelineModel.transform(data)


In [ ]:
finalized_data=data_w_features.select("features",'Grades')

In [ ]:
train_dataset,test_dataset=finalized_data.randomSplit([0.7,0.3])

In [ ]:
from pyspark.ml.regression import LinearRegression
lin=LinearRegression(featuresCol='features',labelCol='Grades')
model=lin.fit(train_dataset)
pred=model.evaluate(test_dataset)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluation=RegressionEvaluator(labelCol='Grades',predictionCol='prediction')

rmse=evaluation.evaluate(pred.predictions,{evaluation.metricName:'rmse'})
print('eval %.3f' %rmse)
mse=evaluation.evaluate(pred.predictions,{evaluation.metricName:'mse'})
print('eval %.3f' %mse)
mae=evaluation.evaluate(pred.predictions,{evaluation.metricName:'mae'})
print('eval %.3f' %mae)
r2=evaluation.evaluate(pred.predictions,{evaluation.metricName:'r2'})
print('eval %.3f' %r2)

In [12]:
from google.colab import files
files.upload()

KeyboardInterrupt: ignored

In [ ]:
data.dtypes[('expense','double')]
data.dtypes[('expense','string')]
data.dtypes[('expense','int')]
data.dtypes[('expense','double')]

categorical_cols=[item[0] for item in data.dtypes if item[1].startswith('string')]
numerical_cols=[item[0] for item in data.dtypes if item[1].startswith('int') | item[1].startwith('double')][:-1]